In [2]:
import numpy as np
import pandas as pd

# Introduction


In the last few years speed dating popularity has grown quickly. Despite its popularity lots of people don't seem as satisfied as they'd like. Most users don't end up finding what they were looking for. That's why a crew of data scientists is going to study data on previous speed dating events in order to make it easier for our users to find their other halves.

**Disclaimer:** Most of the data has been recorded from heterosexual encounters this makes it difficult to inferentiate the data into our system. *(New Speed Dating events are more plural taking into account all sexs and genders)*

# What are we looking for?

### Finding questions

The first thing we have to do is ask ourselfs what conclusions we hope this study leads to. In other words finding the questions this project is going to answer to.

First of all we want to maximize the likelyhood two people fall in love.

>**"Are these two people going to match?"** - *(After selecting two people from a new wave)*

Also for those who were refused we want them classified in a group within all the users. The sooner someone is included into a group of users, the better affinity it will have with those in the same group.

>**"Which group this person belongs to?"**

We don't want any negative encounter. So it may be useful not to let into the Speed Dating any individual which is going to have a negative influence on the event.

>**"Is this person suitable for todays event?"**


"Speed Dating" data tidying
-----
The first thing to do is fix possible errors so that it is easier to approach the solution.

In [30]:
speedDatingDF = pd.read_csv("Speed Dating Data.csv",encoding = "ISO-8859-1")
speedDatingDF.dtypes

iid           int64
id          float64
gender        int64
idg           int64
condtn        int64
wave          int64
round         int64
position      int64
positin1    float64
order         int64
partner       int64
pid         float64
match         int64
int_corr    float64
samerace      int64
age_o       float64
race_o      float64
pf_o_att    float64
pf_o_sin    float64
pf_o_int    float64
pf_o_fun    float64
pf_o_amb    float64
pf_o_sha    float64
dec_o         int64
attr_o      float64
sinc_o      float64
intel_o     float64
fun_o       float64
amb_o       float64
shar_o      float64
             ...   
amb1_3      float64
shar1_3     float64
attr7_3     float64
sinc7_3     float64
intel7_3    float64
fun7_3      float64
amb7_3      float64
shar7_3     float64
attr4_3     float64
sinc4_3     float64
intel4_3    float64
fun4_3      float64
amb4_3      float64
shar4_3     float64
attr2_3     float64
sinc2_3     float64
intel2_3    float64
fun2_3      float64
amb2_3      float64


Every variable is described in the piece of writing called ***"Speed Dating Data.doc"*** which is in the same directory as this notebook.

And in cases such as gender or race we can see using numbers is not the best choice. That's why we are going to modify those variables which are not countable.

In [31]:
speedDatingDF.loc[speedDatingDF['gender'] == 0, 'gender'] = "Female"
speedDatingDF.loc[speedDatingDF['gender'] == 1, 'gender'] = "Male"

In [32]:
speedDatingDF.loc[speedDatingDF['condtn'] == 1, 'condtn'] = "Limited"
speedDatingDF.loc[speedDatingDF['condtn'] == 2, 'condtn'] = "Extensive"

In [33]:
speedDatingDF.loc[speedDatingDF['match'] == 0, 'match'] = False
speedDatingDF.loc[speedDatingDF['match'] == 1, 'match'] = True

In [34]:
speedDatingDF.loc[speedDatingDF['samerace'] == 0, 'samerace'] = False
speedDatingDF.loc[speedDatingDF['samerace'] == 1, 'samerace'] = True

In [51]:
speedDatingDF.loc[speedDatingDF['field_cd'] == 1, 'field_cd'] = "Law"
speedDatingDF.loc[speedDatingDF['field_cd'] == 2, 'field_cd'] = "Math"
speedDatingDF.loc[speedDatingDF['field_cd'] == 3, 'field_cd'] = "Social Science/Psychologist"
speedDatingDF.loc[speedDatingDF['field_cd'] == 4, 'field_cd'] = "Medical Science/Pharmaceuticals/Bio Tech"
speedDatingDF.loc[speedDatingDF['field_cd'] == 5, 'field_cd'] = "Engineering"
speedDatingDF.loc[speedDatingDF['field_cd'] == 6, 'field_cd'] = "English/Creative Writing/Journalism"
speedDatingDF.loc[speedDatingDF['field_cd'] == 7, 'field_cd'] = "History/Religion/Philosophy"
speedDatingDF.loc[speedDatingDF['field_cd'] == 8, 'field_cd'] = "Bussines/Econ/Finance"
speedDatingDF.loc[speedDatingDF['field_cd'] == 9, 'field_cd'] = "Education/Academia"
speedDatingDF.loc[speedDatingDF['field_cd'] == 10, 'field_cd'] = "Biological Sciences/Chemistry/Physics"
speedDatingDF.loc[speedDatingDF['field_cd'] == 11, 'field_cd'] = "Social work"
speedDatingDF.loc[speedDatingDF['field_cd'] == 12, 'field_cd'] = "Undergrad/undecided"
speedDatingDF.loc[speedDatingDF['field_cd'] == 13, 'field_cd'] = "Political Science/International Affairs"
speedDatingDF.loc[speedDatingDF['field_cd'] == 14, 'field_cd'] = "Film"
speedDatingDF.loc[speedDatingDF['field_cd'] == 15, 'field_cd'] = "Fine Arts/Arts administration"
speedDatingDF.loc[speedDatingDF['field_cd'] == 16, 'field_cd'] = "Languages"
speedDatingDF.loc[speedDatingDF['field_cd'] == 17, 'field_cd'] = "Architecture"
speedDatingDF.loc[speedDatingDF['field_cd'] == 18, 'field_cd'] = "Other"

In [52]:
speedDatingDF.loc[speedDatingDF['race'] == 1, 'race'] = "Black/African American"
speedDatingDF.loc[speedDatingDF['race'] == 2, 'race'] = "European/Caucassian-American"
speedDatingDF.loc[speedDatingDF['race'] == 3, 'race'] = "Latino/Hispanic American"
speedDatingDF.loc[speedDatingDF['race'] == 4, 'race'] = "Asian/Pacific Islander/Asian-American"
speedDatingDF.loc[speedDatingDF['race'] == 5, 'race'] = "Native American"
speedDatingDF.loc[speedDatingDF['race'] == 6, 'race'] = "Other"

In [48]:
speedDatingDF.loc[speedDatingDF['length'] == 1, 'length'] = "Too little"
speedDatingDF.loc[speedDatingDF['length'] == 2, 'length'] = "Too much"
speedDatingDF.loc[speedDatingDF['length'] == 3, 'length'] = "Just right"

In [49]:
speedDatingDF.loc[speedDatingDF['numdat_2'] == 1, 'numdat_2'] = "Too little"
speedDatingDF.loc[speedDatingDF['numdat_2'] == 2, 'numdat_2'] = "Too much"
speedDatingDF.loc[speedDatingDF['numdat_2'] == 3, 'numdat_2'] = "Just right"

In [41]:
speedDatingDF.loc[speedDatingDF['date_3'] == 0, 'date_3'] = True
speedDatingDF.loc[speedDatingDF['date_3'] == 1, 'date_3'] = False

In [16]:
speedDatingDF.describe()

/Users/Diario/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,iid,id,idg,condtn,wave,round,position,positin1,order,partner,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,8378.000000,...,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
mean,283.675937,8.960248,17.327166,1.828837,11.350919,16.872046,9.042731,9.295775,8.927668,8.963595,...,7.240312,8.093357,8.388777,7.658782,7.391545,6.810020,7.615079,7.932540,7.155258,7.048611
std,158.583367,5.491329,10.940735,0.376673,5.995903,4.358458,5.514939,5.650199,5.477009,5.491068,...,1.576596,1.610309,1.459094,1.744670,1.961417,1.507341,1.504551,1.340868,1.672787,1.717988
min,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,4.000000,1.000000,1.000000
25%,154.000000,NaN,8.000000,2.000000,7.000000,14.000000,4.000000,NaN,4.000000,4.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,281.000000,NaN,16.000000,2.000000,11.000000,18.000000,8.000000,NaN,8.000000,8.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,407.000000,NaN,26.000000,2.000000,15.000000,20.000000,13.000000,NaN,13.000000,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,552.000000,22.000000,44.000000,2.000000,21.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,12.000000,12.000000,12.000000,12.000000,12.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [53]:
speedDatingDF

0                                                  lawyer
1                                                  lawyer
2                                                  lawyer
3                                                  lawyer
4                                                  lawyer
5                                                  lawyer
6                                                  lawyer
7                                                  lawyer
8                                                  lawyer
9                                                  lawyer
10                                                    law
11                                                    law
12                                                    law
13                                                    law
14                                                    law
15                                                    law
16                                                    law
17            